In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Projeto Final do Módulo SQL

Este notebook apresenta o exercício final do módulo SQL do curso de Data Science da EBAC, no qual devemos apresentar um storytelling sobre a geração de um insight por meio da exploração de um conjunto de dados de crédito disponibilizado aos alunos. 


## Conjunto de dados

O conjunto de dados disponibilizado possui as seguintes informações: 

| Coluna  | Descrição |
| ------- | --------- |
| idade   | Idade do cliente |
| sexo    | Sexo do cliente |
| depedentes | Número de dependentes do cliente |
| escolaridade | Escolaridade do cliente |
| estado_civil | Estado civil do cliente |
| salario_anual | Faixa do salario mensal multiplicado por 12 |
| tipo_cartao | Categoria do cartao: blue, silver, gold e platinium |
| qtd_produtos | Quantidade de produtos contratados |
| iteracoes_12m | Quantidade de iteracoes com o cliente no último ano |
| meses_inativo_12m | Quantidade de meses que o cliente ficou inativo no último ano |
| limite_credito | Valor do limite do cartão de crédito |
| valor_transacoes_12m | Soma total do valor das transações no cartão de crédito no último ano |
| qtd_transacoes_12m | Quantidade total de transações no cartão de crédito no último ano |

## Método

O conjunto de dados disponibilizado foi carregado em um AWS S3 Bucket e será explorado por meio de queries SQL com o uso da ferramenta AWS Athena. 

## Insight

A pergunta que queremos responder é:

**No conjunto de dados disponibilizado, o salário das mulheres é realmente menor que o salário dos homens?**

## Exploração dos Dados

Nesta etapa, vamos verificar e entender o conjunto de dados a ser trabalhado. Vamos iniciar verificando o acesso aos dados. Para isso, vamos executar a query abaixo:

> select * from credito limit 10

![alt](https://github.com/tedymc/curso-ebac-ciencia-dados/blob/main/ebac-sql/Captura%20de%20Tela%202023-10-10%20a%CC%80s%207.26.09%20PM.png?raw=true)

Verificamos que os nomes das colunas coincidem com a descrição dos dados apresentada anteriormente.

### Número de registros

Verificamos também o número de registros presentes no conjunto de dados com a query abaixo:

> select count(*) from credito

![alt](https://github.com/tedymc/curso-ebac-ciencia-dados/blob/main/ebac-sql/Captura%20de%20Tela%202023-10-10%20a%CC%80s%207.24.48%20PM.png?raw=true)

Assim, descobrimos que temos 2654 registros de crédito no conjunto de dados.


### Tipos de dados

É possível verificar o tipo dos dados com o comando abaixo:

> describe credito

![alt](https://github.com/tedymc/curso-ebac-ciencia-dados/blob/main/ebac-sql/Captura%20de%20Tela%202023-10-10%20a%CC%80s%207.33.44%20PM.png?raw=true)

O dado *salario_anual* refere-se a uma faixa de salários. Vamos verificar quais os possíveis valores para essa coluna com a query abaixo:

> select salario_anual, count(*) from credito group by salario_anual

![alt](https://github.com/tedymc/curso-ebac-ciencia-dados/blob/main/ebac-sql/Captura%20de%20Tela%202023-10-10%20a%CC%80s%207.56.06%20PM.png?raw=true)

Observemos a presença do valor **na**, que indica registros que não contêm informação de salário (valor nulo/missing).

Vamos executar a seguinte query para identificar os valores possíveis para *sexo*

> select sexo, count(*) from credito where salario_anual != 'na' group by sexo

![alt](https://github.com/tedymc/curso-ebac-ciencia-dados/blob/main/ebac-sql/Captura%20de%20Tela%202023-10-10%20a%CC%80s%208.42.18%20PM.png?raw=true)

Desse modo, verificamos que há somente 2 valores possíveis para essa coluna: *M* ou *F*, de tal sorte que podemos tratar esse dado como categórico. Também notamos que não há nenhum dado nulo/missing. 

### Analisando os salários por sexo

Para verificar o cruzamento das informações sexo e salário_anual, procedemos com a seguinte query:

> select sexo, salario_anual, count(*) from credito where salario_anual != 'na' group by sexo, salario_anual

![](https://github.com/tedymc/curso-ebac-ciencia-dados/blob/main/ebac-sql/Captura%20de%20Tela%202023-10-10%20a%CC%80s%207.59.45%20PM.png?raw=true)

Nos resultados, é possível verificar que no conjunto de dados disponibilizado:
- Com relação às mulheres, o salário máximo anual é de até 60 mil reais
- Com relação aos homens, o salário máximo anual é de mais de 120 mil reais

E qual seria o número de homens que ganham mais que 60 mil reais? Vamos descobrir a resposta com a query abaixo:

> select count(*) from credito where sexo='M' and salario_anual not in ('na', 'menos que $40K', '$40K - $60K')

![alt](https://github.com/tedymc/curso-ebac-ciencia-dados/blob/main/ebac-sql/Captura%20de%20Tela%202023-10-10%20a%CC%80s%208.09.27%20PM.png?raw=true)

Vemos que, dos 1539 homens da amostra, 1161 (75%) deles ganham mais que 60 mil reais, e 25% ganham menos que 60 mil reais. Já entre as mulheres, 100% delas ganham menos que 60 mil reais. 


### Visualizando os dados

Os dados acima podem ser melhor visualizados no gráfico abaixo:

![alt](https://github.com/tedymc/curso-ebac-ciencia-dados/blob/6ee0bf6582ada2d6f94284944c5261235402e6e4/ebac-sql/grafico.png?raw=true)








## Conclusão

No tópico *Tipos de dados*, descobrimos que o conjunto de dados possui registros de 790 mulheres e 1539 homens. Já no tópico 'Analisando os salários por sexo', descobrimos que, na amostra utilizada, todas as 790 mulheres ganham menos que 60 mil reais. Já entre os 1539 homens, 1161 (ou 75%) ganham mais que 60 mil reais, e somente 378 (ou 25%) ganham menos que 60 mil reais. É um resultado muito expressivo, o qual nos permite afirmar que **sim**, de acordo com esse conjunto de dados, os homens ganham significativamente mais que as mulheres.